In [1]:
import os, sys
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)
#print(cluster.job_script())
w = cluster.start_workers(15)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.148.1.0:55322 Dashboard: http://10.148.1.0:8787/status,Cluster Workers: 30 Cores: 120 Memory: 1500.00 GB


--- 

# automatic rewriting of all variables (for spectral calculations)

# transposed data: (i,j,time), 1 file per face, time windows

In [5]:
V = ['Eta', 'SST', 'SSS', 'SSU', 'SSV']
V = ['SSS']

transpose = True # False untested

if transpose:
    Nt = 24*10*6 # time windows to consider
    out_dir = scratch+'/mit_nc_new/'
    fsize_bound = 15*1e9*6
else:
    Nt = 1
    out_dir = scratch+'/mit_nc/'    
    fsize_bound = 60*1e6    

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    it = np.arange(time.size/Nt-1).astype(int)*Nt
    it = np.arange(1).astype(int)*Nt # tmp
    assert it[-1]+Nt<time.size
    #
    p = 'C'
    if v is 'SSU':
        p = 'W'
    elif v is 'SSV':
        p = 'S'
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    #ds = ds.chunk({'face': 1})
    #
    for face in range(ds['face'].size):
        for i, t in enumerate(it):
            #
            file_out = out_dir+'/%s_f%02d_t%02d.nc'%(v,face,i)
            if not os.path.isfile(file_out) or os.path.getsize(file_out) < fsize_bound:            
                dv = ds[v].isel(time=slice(t,t+Nt), face=face) 
                # should store grid data independantly in a single file
                dv = dv.drop(['XC','YC','Depth','rA']).to_dataset()
                #
                if transpose:
                    dv = dv.chunk({'time': dv['time'].size, 'i': 432, 'j': 432})
                    dv = dv.transpose('i','j','time')
                    chunksizes = [432, 432, dv['time'].size]
                else:
                    dv = dv.chunk({'i': 432, 'j': 432})
                    chunksizes = [1, 432, 432]
                #print(dv)
                #
                while True:
                    try:
                        %time dv.to_netcdf(file_out, mode='w', unlimited_dims=['time'], \
                                           encoding={'SSS': {'chunksizes': chunksizes}})
                    except:
                        print('Failure')
                    if os.path.isfile(file_out) and os.path.getsize(file_out)>fsize_bound:
                        #
                        print('face=%d / i=%d'%(face,i))
                        break
            else:
                print('face=%d / i=%d - allready processed'%(face,i))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler inst

CPU times: user 2min 20s, sys: 8 s, total: 2min 28s
Wall time: 12min 25s
face=0 / i=0


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 2min 6s, sys: 7 s, total: 2min 13s
Wall time: 11min 52s
face=1 / i=0


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


...

CPU times: user 2min 8s, sys: 7.22 s, total: 2min 15s
Wall time: 12min 5s
face=10 / i=0


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 2min 8s, sys: 7.6 s, total: 2min 15s
Wall time: 12min 6s
face=11 / i=0


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 2min 8s, sys: 7.55 s, total: 2min 15s
Wall time: 12min 5s
face=12 / i=0


--- 

# standard data layout: (face, j, i), 1 file per time (for movies)

In [5]:
V = ['Eta', 'SST', 'SSS', 'SSU', 'SSV']
V = ['SSS']

Nt = 1
#out_dir = datawork+'/mit_nc/'
out_dir = scratch+'/mit_nc_t/'    
fsize_bound = 13*60*1e6

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    it = np.arange(time.size/Nt-1).astype(int)*Nt
    it = np.arange(4).astype(int)*Nt # tmp
    assert it[-1]+Nt<time.size
    #
    p = 'C'
    if v is 'SSU':
        p = 'W'
    elif v is 'SSV':
        p = 'S'
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    #ds = ds.chunk({'face': 1})
    #
    for i, t in enumerate(it):
        #
        file_out = out_dir+'/%s_t%04d.nc'%(v,i)
        if not os.path.isfile(file_out) or os.path.getsize(file_out) < fsize_bound:            
            dv = ds[v].isel(time=slice(t,t+Nt)) 
            # should store grid data independantly in a single file
            dv = dv.drop(['XC','YC','Depth','rA']).to_dataset()
            #
            #print(dv)
            #
            while True:
                try:
                    #print(dv)
                    %time dv.to_netcdf(file_out, mode='w')                    
                except:
                    print('Failure')
                if os.path.isfile(file_out) and os.path.getsize(file_out) > fsize_bound:
                    #
                    print('i=%d, iter=%d'%(i, iters[i].values))
                    break
        else:
            print('i=%d, iter=%d - allready processed'%(i, iters[i].values))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler inst

CPU times: user 716 ms, sys: 36 ms, total: 752 ms
Wall time: 9.69 s
i=0, iter=228096


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 568 ms, sys: 52 ms, total: 620 ms
Wall time: 5.18 s
i=1, iter=228240


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 584 ms, sys: 36 ms, total: 620 ms
Wall time: 5.24 s
i=2, iter=228384


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 584 ms, sys: 32 ms, total: 616 ms
Wall time: 5.05 s
i=3, iter=228528


In [7]:
cluster.scheduler.close()

<Future finished result=None>